# modAL + pyhard- Comparando estratégias

- Amostra por incerteza
- Amostragem aleatória
- Consulta por comitê
- Aprendizado passivo
- Redução do erro esperado

In [1]:
%run -i set_environment

## Bibliotecas

In [37]:
%run -i importing_libraries

## Classificadores

### Algoritmos

In [3]:
%run -i classifiers

### Conjunto de dados

In [4]:
%run -i importing_datasets

## Estatratégias

### Amostra por incerteza

In [5]:
def uncertain_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost):
    
    from modAL.uncertainty import classifier_uncertainty
    
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()
    
    # parte randomica inicial da estratégia
    #initial_idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    #X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][initial_idx]], y_raw[idx_data[idx_bag][TRAIN][initial_idx]]
    #X_test, y_test = X_raw[idx_data[idx_bag][TEST]], y_raw[idx_data[idx_bag][TEST]]
    
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    
    sample_size = sample_size + len(X_train)

    #cls = which_classifier(classifier)
    #cls.fit(X_train,y_train)

    learner = ActiveLearner (
        estimator= which_classifier(classifier), #cls,
        query_strategy=uncertainty_sampling,
        X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
    )
    
    uncertain_sample_score = learner.score(X_test, y_test)
    performance_history.append(uncertain_sample_score)

    total_of_samples = 1
    while (total_of_samples != cost):
        
        #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)
        
        idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
        X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]
        
        if classifier_uncertainty(learner, X_train[0].reshape(1,-1)) > 0.2:
            #print("IF", learner.score(X_test, y_test))
            sample_size = sample_size + len(X_train)
            learner.teach(X_train, y_train)
            uncertain_sample_score = learner.score(X_test, y_test)
            performance_history.append(uncertain_sample_score)
        total_of_samples = total_of_samples + 1
    
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw), # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
             "Strategy": "Uncertain Sampling"}

### Amostragem aleatória

In [6]:
def random_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost):
        
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()

    for i in range(1, cost+1):

        #high = X_raw.shape[0] = qtd amostras no dataset
        #training_indices = np.random.randint(low=0, high=len(X_raw[idx_data[idx_bag][TRAIN]]), size=k+i) #high = qtd elementos na bag
        #sample_size = sample_size + len(training_indices)
        #X_train = X_raw[idx_data[idx_bag][TRAIN][training_indices]] #ASK06
        #y_train = y_raw[idx_data[idx_bag][TRAIN][training_indices]]
        #X_test = np.delete(X_raw, idx_data[idx_bag][TRAIN][training_indices], axis=0)
        #y_test = np.delete(y_raw, idx_data[idx_bag][TRAIN][training_indices], axis=0)

        X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
        sample_size = sample_size + len(X_train)
        
        cls = which_classifier(classifier)
        cls.fit(X_train, y_train)

        random_sampling_score = cls.score(X_test,y_test)
        performance_history.append(random_sampling_score)

        
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw),
             "Strategy": "Random Sampling"}

### Consulta por comitê

In [7]:
def query_by_committee(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost):

    from modAL.models import ActiveLearner, Committee
    from modAL.disagreement import vote_entropy_sampling

    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()

    learner_list = []

    for j in range(1, cost+1): # Loop para criação do comitê

        X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
        sample_size = sample_size + len(X_train)

        # initializing learner
        learner = ActiveLearner(
            estimator= which_classifier(classifier),
            X_training = X_train, y_training = y_train 
        )
        learner_list.append(learner)

    # assembling the committee
    committee = Committee(
        learner_list=learner_list,
        query_strategy=vote_entropy_sampling)

    #X_pool, y_pool = X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]]
    
    # query by committee
    for idx in range(cost):
        print("\t Size of X_pool:", len(X_pool))
        query_idx, query_instance = committee.query(X_pool, n_instances = init_size+1)
        sample_size = sample_size + len(query_idx)
        
        committee.teach(
            X = X_pool[query_idx],
            y = y_pool[query_idx]
        )

        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx)
        
        query_by_committee_score = committee.score(X_pool, y_pool)
        performance_history.append(query_by_committee_score)

        
    end = timer()
    time_elapsed = end - start

    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw),
             "Strategy": "Query by Committee"}

### Expected Error Reduction

In [80]:
def exp_error_reduction(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost):

    from modAL.expected_error import expected_error_reduction
    
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()
    
    # parte randomica inicial da estratégia
    #initial_idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    #X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][initial_idx]], y_raw[idx_data[idx_bag][TRAIN][initial_idx]]
    #X_pool, y_pool = X_raw[idx_data[idx_bag][TEST]], y_raw[idx_data[idx_bag][TEST]]
    
    X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    sample_size = sample_size + len(X_train)

    X_pool, y_pool = X_raw[idx_data[idx_bag][TEST]], y_raw[idx_data[idx_bag][TEST]]
    
    learner = ActiveLearner (
        estimator = which_classifier(classifier),
        X_training = X_train, y_training = y_train
    )
    exp_er_score = learner.score(X_pool, y_pool)
    performance_history.append(exp_er_score)

    total_of_samples = 1
    while (total_of_samples != cost):
        print("\t Size of X_pool:", len(X_pool))
        exp_error_idx = expected_error_reduction(learner, X_pool, 'binary', n_instances=init_size)

        learner.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
        sample_size = sample_size + init_size
    
        X_pool = np.delete(X_pool, exp_error_idx, axis=0)
        y_pool = np.delete(y_pool, exp_error_idx)
        
        exp_er_score = learner.score(X_pool, y_pool)
        performance_history.append(exp_er_score)
        
        total_of_samples = total_of_samples + 1
    
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw), # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
             "Strategy": "Expected Error Reduction"}

### Expected Model Change

In [9]:
def exp_model_change(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost):

    from modAL.expected_error import expected_error_reduction
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()
    
    # parte randomica inicial da estratégia
    #initial_idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    #X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][initial_idx]], y_raw[idx_data[idx_bag][0][initial_idx]]
    #X_pool, y_pool = X_raw[idx_data[idx_bag][TEST]], y_raw[idx_data[idx_bag][TEST]]
    
    X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    sample_size = sample_size + len(X_train)

    learner = ActiveLearner (
        estimator = which_classifier(classifier),
        X_training = X_train, y_training = y_train
    )
    
#     performance_history.append(uncertain_sample_score)

    total_of_samples = 1
    while (total_of_samples != cost):
        print("\t Size of X_pool:", len(X_pool))
        exp_error_idx = np.random.choice(range(len(X_pool)), size=init_size, replace=False)[0]
        aux = deepcopy(learner)

        aux.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
        score_aux = aux.score(X_pool, y_pool)
        score_learner = learner.score(X_pool, y_pool)

        if score_aux > score_learner:
            learner = deepcopy(aux)
            sample_size = sample_size + init_size
        
        X_pool = np.delete(X_pool, exp_error_idx, axis=0)
        y_pool = np.delete(y_pool, exp_error_idx, axis=0)
        
        exp_mo_score = learner.score(X_pool, y_pool)
        performance_history.append(exp_mo_score)

        total_of_samples = total_of_samples + 1
    
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw), # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
             "Strategy": "Expected Model Change"}

## Pyhard Strategies

In [71]:
def config(section, filename='strategies.config'):
    from configparser import ConfigParser

    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read("../" + filename)
    # get section, default to postgresql
    strategy = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            strategy[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    # transformando texto em bool
    strategy['ascending'] = list(map(lambda x: bool(0 if x == "False" else 1), strategy['ascending'].split(',')))
    strategy['sortby'] = strategy['sortby'].split(',')
    
    print(strategy)
    
    return strategy

In [74]:
def pyhard_strategies(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy):
    
    from modAL.uncertainty import classifier_uncertainty
    
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()
    
    strategy = config(strategy)
    
    # parte randomica inicial da estratégia
    
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    
    sample_size = sample_size + len(X_train)

    learner = ActiveLearner (
        estimator= which_classifier(classifier), #cls,
        query_strategy=uncertainty_sampling,
        X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
    )
    
    uncertain_sample_score = learner.score(X_test, y_test)
    performance_history.append(uncertain_sample_score)

    total_of_samples = 1

    #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)

    idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]

    X_rawAndY_raw = np.column_stack([X_raw[idx_data[idx_bag][TRAIN]],y_raw[idx_data[idx_bag][TRAIN]]])
    np.savetxt("data.csv", X_rawAndY_raw, fmt='%i', delimiter=",")
    
    which_pyhard_measure(strategy['measure'])

    !pyhard --no-isa

    df = pd.read_csv('metadata.csv')

    idx = list(df.sort_values(by=strategy['sortby'], ascending=strategy['ascending'])['instances'][:cost])

    X_train = X_raw[idx_data[idx_bag][TRAIN][idx]]
    y_train = y_raw[idx_data[idx_bag][TRAIN][idx]]

    sample_size = cost
    learner.teach(X_train, y_train)
    uncertain_sample_score = learner.score(X_test, y_test)
    performance_history.append(uncertain_sample_score)
    
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history[-1], 
             "time_elapsed": time_elapsed,
             "classifier": classifier,
             "sample_size": sample_size / len(X_raw), # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
             "Strategy": strategy['name']}

In [75]:
pyhard_strategies_names = ['H','U','H+U','LSC','N2','F3']

for ds in datasets:
    for classifier in classifiers:
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
        #para cada i em idx_bag ("n_splits") (1 a 5)
        for idx_bag in range(n_splits):
            for ph_strategy in pyhard_strategies_names:
                tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " " + ph_strategy)
                result = pyhard_strategies(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost, ph_strategy)
                result['dataset'] = ds[:-5]
                total_performance_history.append(result)
                tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " " + ph_strategy)        

Testando: 61_iris 5NN 0/5 H
{'name': 'Lowest Harmfulness Sampling', 'measure': 'Harmfulness', 'sortby': ['feature_Harmfulness'], 'ascending': [True]}
run 'pyhard --help' to see all options.
[INFO] 2021-04-26 21:02:14,889 - Configuration file: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/config.yaml'
[INFO] 2021-04-26 21:02:14,897 - Reading input dataset: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/data.csv'
[INFO] 2021-04-26 21:02:14,907 - Type of problem: 'classification'
[INFO] 2021-04-26 21:02:14,907 - Building metadata.
[INFO] 2021-04-26 21:02:19,740 - Calculating measure 'Harmfulness'
[INFO] 2021-04-26 21:02:19,743 - Assessing performance of classifier 'random_forest'
[INFO] 2021-04-26 21:02:19,743 - Estimating instance performance...
[INFO] 2021-04-26 21:02:19,745 - Evaluating testing fold #1
[INFO] 2021-04-26 21:02:20,432 - Test fold mean accuracy: 0.9615384615384616
[INF

[INFO] 2021-04-26 21:06:48,916 - Test fold mean accuracy: 0.9423076923076923
[INFO] 2021-04-26 21:06:48,916 - Iteration 1/1 completed.
[INFO] 2021-04-26 21:06:48,916 - Mean accuracy on test instances (iteration #1): 0.9519
[INFO] 2021-04-26 21:06:48,970 - Total elapsed time: 6.8s
[INFO] 2021-04-26 21:06:48,971 - Instance Hardness analysis finished.
Passou: 61_iris 5NN 0/5 F3
Testando: 61_iris 5NN 1/5 H
{'name': 'Lowest Harmfulness Sampling', 'measure': 'Harmfulness', 'sortby': ['feature_Harmfulness'], 'ascending': [True]}
run 'pyhard --help' to see all options.
[INFO] 2021-04-26 21:07:38,760 - Configuration file: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/config.yaml'
[INFO] 2021-04-26 21:07:38,766 - Reading input dataset: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/data.csv'
[INFO] 2021-04-26 21:07:38,775 - Type of problem: 'classification'
[INFO] 2021-04-26 21:07:38,776 - Bu

[INFO] 2021-04-26 21:12:05,517 - Assessing performance of classifier 'random_forest'
[INFO] 2021-04-26 21:12:05,517 - Estimating instance performance...
[INFO] 2021-04-26 21:12:05,519 - Evaluating testing fold #1
[INFO] 2021-04-26 21:12:06,328 - Test fold mean accuracy: 0.9807692307692307
[INFO] 2021-04-26 21:12:06,328 - Evaluating testing fold #2
[INFO] 2021-04-26 21:12:07,025 - Test fold mean accuracy: 0.9423076923076923
[INFO] 2021-04-26 21:12:07,025 - Iteration 1/1 completed.
[INFO] 2021-04-26 21:12:07,026 - Mean accuracy on test instances (iteration #1): 0.9615
[INFO] 2021-04-26 21:12:07,084 - Total elapsed time: 6.4s
[INFO] 2021-04-26 21:12:07,084 - Instance Hardness analysis finished.
Passou: 61_iris 5NN 1/5 F3
Testando: 61_iris 5NN 2/5 H
{'name': 'Lowest Harmfulness Sampling', 'measure': 'Harmfulness', 'sortby': ['feature_Harmfulness'], 'ascending': [True]}
run 'pyhard --help' to see all options.
[INFO] 2021-04-26 21:12:49,126 - Configuration file: '/mnt/c/Users/ahmou/OneDrive/

[INFO] 2021-04-26 21:16:54,966 - Reading input dataset: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/data.csv'
[INFO] 2021-04-26 21:16:55,009 - Type of problem: 'classification'
[INFO] 2021-04-26 21:16:55,010 - Building metadata.
[INFO] 2021-04-26 21:17:06,249 - Calculating measure 'F3'
[INFO] 2021-04-26 21:17:06,460 - Assessing performance of classifier 'random_forest'
[INFO] 2021-04-26 21:17:06,461 - Estimating instance performance...
[INFO] 2021-04-26 21:17:06,467 - Evaluating testing fold #1
[INFO] 2021-04-26 21:17:07,702 - Test fold mean accuracy: 0.9230769230769231
[INFO] 2021-04-26 21:17:07,703 - Evaluating testing fold #2
[INFO] 2021-04-26 21:17:08,879 - Test fold mean accuracy: 0.9423076923076923
[INFO] 2021-04-26 21:17:08,879 - Iteration 1/1 completed.
[INFO] 2021-04-26 21:17:08,880 - Mean accuracy on test instances (iteration #1): 0.9327
[INFO] 2021-04-26 21:17:08,961 - Total elapsed time: 14.1s
[INFO] 2021-04-26 21:17:08

Passou: 61_iris 5NN 3/5 N2
Testando: 61_iris 5NN 3/5 F3
{'name': 'Lowest F3 Sampling', 'measure': 'F3', 'sortby': ['feature_F3'], 'ascending': [True]}
run 'pyhard --help' to see all options.
[INFO] 2021-04-26 21:22:02,300 - Configuration file: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/config.yaml'
[INFO] 2021-04-26 21:22:02,306 - Reading input dataset: '/mnt/c/Users/ahmou/OneDrive/Documentos/ubuntu_wd/act_len/active_learning/data/act_len labs/pyhard/data.csv'
[INFO] 2021-04-26 21:22:02,326 - Type of problem: 'classification'
[INFO] 2021-04-26 21:22:02,327 - Building metadata.
[INFO] 2021-04-26 21:22:11,640 - Calculating measure 'F3'
[INFO] 2021-04-26 21:22:11,927 - Assessing performance of classifier 'random_forest'
[INFO] 2021-04-26 21:22:11,927 - Estimating instance performance...
[INFO] 2021-04-26 21:22:11,930 - Evaluating testing fold #1
[INFO] 2021-04-26 21:22:13,576 - Test fold mean accuracy: 0.9615384615384616
[INFO] 2021-

KeyError: 'feature_LSC'

## Setup

In [18]:
def which_pyhard_measure(measure='LSC'):
    import yaml
    with open(r'config-template.yaml') as file:
        configs_list = yaml.load(file, Loader=yaml.FullLoader)

        if measure == 'LSC':
            configs_list['measures_list'] = ['LSC']
        elif measure == 'Harmfulness':
            configs_list['measures_list'] = ['Harmfulness']
        elif measure == 'Usefulness':
            configs_list['measures_list'] = ['Usefulness']
        elif measure == 'U+H':
            configs_list['measures_list'] = ['Harmfulness','Usefulness']
        elif measure == 'N2':
            configs_list['measures_list'] = ['N2']
        elif measure == 'F3':
            configs_list['measures_list'] = ['F3']

    with open(r'config.yaml', 'w') as file:
        yaml.dump(configs_list, file)

In [19]:
def which_dataset(dataset = "iris", n_splits = 5):
    
    # Futuramente essa etapa será ajustada para receber qualquer dataset (ou lista com datasets)
    if (dataset == "iris"):
        data = load_iris()
        X_raw = data['data']
        y_raw = data['target']
    
    if (dataset == "wine"):
        data = load_wine()
        X_raw = data['data']
        y_raw = data['target']
        
    if (dataset == "digits"):
        data = load_digits()
        X_raw = data['data']
        y_raw = data['target']
        
    # cross validation bags
    data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=0.7, random_state=0) #n_splits
    
    # extraindo ids do data_cv
    idx_data = []
    for train_index, test_index in data_cv.split(X_raw):
            idx_data.append([train_index, test_index])

    return X_raw, y_raw, idx_data

In [20]:
def which_oml_dataset(dataset_id, n_splits = 5):
    data = openml.datasets.get_dataset(dataset_id)
    
    X_raw, y_raw, categorical_indicator, attribute_names = data.get_data(
    dataset_format="array", target=data.default_target_attribute)
    
    le = preprocessing.LabelEncoder()
    le.fit(y_raw)
    y_raw = le.transform(y_raw)
    
    X_raw = np.nan_to_num(X_raw)
    
    data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=0.7, random_state=0) #n_splits
    
    idx_data = []
    for train_index, test_index in data_cv.split(X_raw):
            idx_data.append([train_index, test_index])

    return X_raw, y_raw, idx_data, data.name

In [21]:
def which_arff_dataset(dataset, n_splits = 5):
   
    from sklearn.preprocessing import OrdinalEncoder
    
    data = arff.loadarff('datasets/luis/' + dataset)
    data = pd.DataFrame(data[0])

    X_raw = data[data.columns[:-1]].to_numpy()
    y_raw = data[data.columns[-1]].to_numpy()
    
    lex = preprocessing.OrdinalEncoder()
    lex.fit(X_raw)
    X_raw = lex.transform(X_raw)
        
    ley = preprocessing.LabelEncoder()
    ley.fit(y_raw)
    y_raw = ley.transform(y_raw)
    
    # cross validation bags
    data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=0.7, random_state=0) #n_splits
    data_cv.get_n_splits(X_raw,y_raw)
    
    # extraindo ids do data_cv
    idx_data = []
    for train_index, test_index in data_cv.split(X_raw, y_raw):
            idx_data.append([train_index, test_index])

    return X_raw, y_raw, idx_data, dataset

In [22]:
def which_classifier(classifier = '5NN'):
    
    if (classifier == '5NN'):
        return KNeighborsClassifier(5)
    elif (classifier == 'C4.5'):
        return tree.DecisionTreeClassifier()
    elif (classifier == 'NB'):
        return GaussianNB()
    elif (classifier == 'SVM'):
        return SVC(probability=True, gamma='auto')
    elif (classifier == 'RF'):
        return RandomForestClassifier()

In [23]:
def fetch_datasets(dataset):
    
    data = arff.loadarff('./datasets/luis/' + dataset)
    metadata = data[1]
    data = pd.DataFrame(data[0])
    
    instances = len(data)
    classes = len(data.iloc[:,-1].value_counts())
    attributes = len(data.columns)- 1
    nominal_attributes = str(metadata).count("nominal")
    
    proportion = data.iloc[:,-1].value_counts()
    proportion = proportion.map(lambda x: round(x/instances*100,2))

    majority = max(proportion)
    minority = min(proportion)

    
    return {
        "name": dataset[:-5],
        "instances": instances,
        "classes": classes,
        "attributes": attributes,
        "nominal attributes": nominal_attributes,
        "majority": majority,
        "minority": minority
    }

In [24]:
datasets = os.listdir('./datasets/luis')
classifiers = ['5NN', 'C4.5', 'NB','RF']
total_performance_history = []

In [25]:
datasets

['61_iris.arff']

In [26]:
metadata = []

for ds in datasets:
    metadata.append(fetch_datasets(ds))

metadata = pd.DataFrame.from_dict(metadata)
metadata

NameError: name 'arff' is not defined

In [83]:
functions = ["uncertain_sampling","random_sampling","query_by_committee","exp_error_reduction","exp_model_change"]
parameters = "(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)"

for ds in tqdm(datasets,  desc ="Dataset"):
    for classifier in tqdm(classifiers,  desc ="Classifier"):
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
        #para cada i em idx_bag ("n_splits") (1 a 5)
        for idx_bag in tqdm(range(n_splits),  desc ="Bag"):
            for func in functions:
                tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + func)
                result = eval(func+parameters)
                result['dataset'] = ds[:-5]
                total_performance_history.append(result)
                tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + func)

Classifier:   0%|          | 0/4 [00:00<?, ?it/s]

Bag:   0%|          | 0/5 [00:00<?, ?it/s]

                                    
                                                 

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]/s]

                                    
                                                 

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]/s]

                                    
                                                 

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]/s]

Testando: 61_iris 5NN 0/5 uncertain_sampling
Passou: 61_iris 5NN 1/5 uncertain_sampling
Testando: 61_iris 5NN 0/5 random_sampling




                                    
                                                 

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]/s]

                                    
                                                 

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]/s]

Passou: 61_iris 5NN 1/5 random_sampling
Testando: 61_iris 5NN 0/5 query_by_committee
	 Size of X_pool: 97
	 Size of X_pool: 91
	 Size of X_pool: 85
	 Size of X_pool: 79
	 Size of X_pool: 73
	 Size of X_pool: 67
	 Size of X_pool: 61
	 Size of X_pool: 55
	 Size of X_pool: 49
	 Size of X_pool: 43




                                    
                                                 

Dataset:   0%|          | 0/1 [00:01<?, ?it/s]/s]

                                    
                                                 

Dataset:   0%|          | 0/1 [00:01<?, ?it/s]/s]

Passou: 61_iris 5NN 1/5 query_by_committee
Testando: 61_iris 5NN 0/5 exp_error_reduction
	 Size of X_pool: 45
	 Size of X_pool: 40
	 Size of X_pool: 35
	 Size of X_pool: 30
	 Size of X_pool: 25
	 Size of X_pool: 20


Dataset:   0%|          | 0/1 [00:03<?, ?it/s]

	 Size of X_pool: 15
	 Size of X_pool: 10
	 Size of X_pool: 5


ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.

In [ ]:
classifiers = ['RF']
total_performance_history = []

In [ ]:
#tqdm_datasets = tqdm(datasets, desc=" Dataset: "+ str(ds[:-5]))
#tqdm_classifier = tqdm(classifiers, desc="Classifier: "+ str(classifier))

for ds in tqdm(datasets,  desc ="Dataset"):
    for classifier in tqdm(classifiers,  desc ="Classifier"):
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
        

        #para cada i em idx_bag ("n_splits") (1 a 5)
        for idx_bag in tqdm(range(n_splits),  desc ="Bag"):

            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_f3_sampling")
            result = lowest_f3_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_f3_sampling")
            
            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " highest_lsc_sampling")
            result = highest_lsc_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " highest_lsc_sampling")
            
            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " highest_usefulness_sampling")
            result = highest_usefulness_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " highest_usefulness_sampling")
           
            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_harmfulness_sampling")
            result = lowest_harmfulness_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_harmfulness_sampling")
            
            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_h_highest_u_sampling")
            result = lowest_h_highest_u_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_h_highest_u_sampling")
  
            tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_n2_sampling")
            result = lowest_n2_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, cost)
            result['dataset'] = ds[:-5]
            total_performance_history.append(result)
            tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " lowest_n2_sampling")

In [ ]:
total_performance_history

## Visualization

### Preprocessing

In [ ]:
df = pd.DataFrame.from_dict(total_performance_history)

In [ ]:
df = df.explode('performance_history')

In [ ]:
df[df.Strategy != "Query by Committee"].sort_values('performance_history', ascending = False)

In [ ]:
df[df.Strategy == "Expected Error Reduction"].sort_values('time_elapsed', ascending = False)

In [ ]:
df.info()

### Plots

In [ ]:
g = sns.relplot(
    data= df,
    x="performance_history", y="time_elapsed",
    hue="Strategy", size="sample_size",
    palette=sns.color_palette(n_colors=5), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

In [ ]:
g = sns.relplot(
    data= df[(df.Strategy != "Uncertain Sampling") & (df.Strategy != "Query by Committee")],
    x="performance_history", y="time_elapsed",
    hue="Strategy", size="sample_size",
    palette=sns.color_palette(n_colors=3), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

In [ ]:
g = sns.relplot(
    data= df[(df.Strategy == "Uncertain Sampling") | (df.Strategy == "Query by Committee")],
    x="performance_history", y="time_elapsed",
    hue="Strategy", size="sample_size",
    palette=sns.color_palette(n_colors=2), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

## Baixando datasets

In [ ]:
from tqdm.notebook import tqdm, trange
p_bar = tqdm(datalist)
for dataset_id in p_bar:
    X_raw, y_raw, idx_data, dataset_name = which_oml_dataset(dataset_id)
    p_bar.set_description(f'"{dataset_name}"')

In [ ]:
ds = "1465_breast-tissue.arff"

X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
   
from modAL.uncertainty import classifier_uncertainty

print(len(np.unique(y_raw)))
X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)), stratify = y_raw[idx_data[idx_bag][TRAIN]])
print(y_train)

learner = ActiveLearner (
    estimator= which_classifier(classifier), #cls,
    query_strategy=uncertainty_sampling,
    X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
)

uncertain_sample_score = learner.score(X_test, y_test)

total_of_samples = 1
while (total_of_samples != cost):

    #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)

    idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]

    if classifier_uncertainty(learner, X_train[0].reshape(1,-1)) > 0.2:
        #print("IF", learner.score(X_test, y_test))
        learner.teach(X_train, y_train)
        uncertain_sample_score = learner.score(X_test, y_test)
        performance_history.append(uncertain_sample_score)
    total_of_samples = total_of_samples + 1


In [ ]:
train_size= len(np.unique(y_raw)) + init_size